In [100]:
from moss.mosaic import Mosaic
import nibabel as nib
import multiprocessing
#os and i/o
import os
import numpy as np
import os.path as op
import seaborn as sns
import matplotlib
import scipy
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [101]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)

In [536]:
exp = 'sim_4mm-onebeta'
masks = ['accumbens_sim',#'hipp','parahipp','entorhinal',
         'V1_fill_sim','cortex_fill_sim',
         'lOFC_fill_sim','mOFC_fill_sim']

In [537]:
# all_rsas = []
# for sub in subs:
#     sub_path = op.abspath('./analysis/' + exp + '/' + sub + '/reg/epi/unsmoothed/')

#     m = masks[0]
#     mask = op.abspath('./data/' + sub + '/masks/' + m +'.nii.gz')
#     mask = nib.load(mask).get_data().astype(bool)

#     rsa = []
#     for run in map(str,range(1,4)):
#         run_dir = sub_path + '/run_'  + run
#         if os.path.exists(run_dir):
#             stats = []
            
#             for i in [1,2,3,4]:
#                 f = run_dir + '/cope' + str(i) + '_xfm.nii.gz'
#                 var = run_dir + '/varcope' + str(i) + '_xfm.nii.gz'
                
#                 var = nib.load(var).get_data().astype(float)
#                 var = var[mask]
#                 std = np.sqrt(var)
                
#                 stat = nib.load(f).get_data().astype(float)
#                 stat = stat[mask]
                
#                 weighted = stat / std
                
#                 stats.append(weighted)
                
#             stats = np.array(stats)
#             rsa.append(np.corrcoef(stats))
            
#     rsa = np.array(rsa)
#     all_rsas.append(np.nanmean(rsa,axis=0))

In [ ]:
roi_rsa = {}
for m in masks:
    all_rsas = []
    for sub in subs:
        sub_path = op.abspath('./analysis/' + exp + '/' + sub + '/reg/epi/smoothed/')

        mask = op.abspath('./data/' + sub + '/masks/' + m +'.nii.gz')
        mask = nib.load(mask).get_data().astype(bool)

        runs = []
        rsa = []
        for run in map(str,range(1,4)):
            run_dir = sub_path + '/run_'  + run
            if os.path.exists(run_dir):
                stats = []

                for i in [1,2,3,4]:
                    f = run_dir + '/cope' + str(i) + '_xfm.nii.gz'
                    var = run_dir + '/varcope' + str(i) + '_xfm.nii.gz'

                    var = nib.load(var).get_data().astype(float)

                    var = var[mask]
                    std = np.sqrt(var)

                    stat = nib.load(f).get_data().astype(float)
                    stat = stat[mask]

                    weighted = stat / std

                    stats.append(weighted)

                stats = np.array(stats)
                runs.append(stats)

        runs = np.array(runs)
        rsa = between_run_correlation(runs)
        all_rsas.append(rsa)

    #make figures
    rsa_norm, fig_mask = get_mean_rsa(all_rsas)
    make_fig(rsa_norm, fig_mask, m)
    make_sig_fig(all_rsas,fig_mask, m)
    roi_rsa[m] = all_rsas
    print 'done',m

In [ ]:
def between_run_correlation(runs):
    nconds = 4
    nruns = runs.shape[0]
    rsa = np.zeros((nconds,nconds))

    for i in range(0,nconds):
        for j in range(0,nconds):
            corr = []

            corr.append(scipy.stats.pearsonr(runs[0,i,:],runs[1,j,:])[0])
            corr.append(scipy.stats.pearsonr(runs[1,i,:],runs[0,j,:])[0])

            if nruns == 3:
                corr.append(scipy.stats.pearsonr(runs[0,i,:],runs[2,j,:])[0])
                corr.append(scipy.stats.pearsonr(runs[1,i,:],runs[2,j,:])[0])
                corr.append(scipy.stats.pearsonr(runs[2,i,:],runs[0,j,:])[0])
                corr.append(scipy.stats.pearsonr(runs[2,i,:],runs[1,j,:])[0])

            rsa[i,j]  = np.mean(corr)
    return rsa

In [ ]:
def get_mean_rsa(all_rsas):
    all_rsas = np.array(all_rsas)
    mean_rsa = np.nanmean(all_rsas,axis=0)
    
    mask = np.zeros_like(mean_rsa) + 1
    mask = np.triu(mask,k=1).astype(int)

    lower = np.tril(mean_rsa)
    lower[lower == 0] = np.nan
    rsa_norm = (mean_rsa - np.nanmean(lower)) / np.nanstd(lower)
    
    return rsa_norm, mask

In [ ]:
def make_fig(rsa_norm, mask,m):
    conds = ['AB+','AC-','B-','C+']
    df = {'AB+': rsa_norm[:,0],
         'AC-': rsa_norm[:,1],
         'B-': rsa_norm[:,2],
         'C+': rsa_norm[:,3]}

    df = pd.DataFrame(df,index = conds)
    df.sort(columns = ['AB+','AC-','B-','C+'])
    df = df[conds]

    with sns.axes_style("white"):
        ax = sns.heatmap(df, mask= mask,vmax = np.max(rsa_norm), square = True, cmap = 'Blues')
    plt.savefig('/home/iballard/Dropbox/fd/figs/' + m + '_smoothed.png',dpi=200)


In [ ]:
def make_sig_fig(all_rsas,mask,m):
    t,p = scipy.stats.ttest_1samp(np.arctanh(all_rsas),0)
    with sns.axes_style("white"):
        ax = sns.heatmap(p, mask = mask, vmax = np.max(p), square = True, cmap = 'Blues_r')
    plt.savefig('/home/iballard/Dropbox/fd/figs/' + m + '_smoothed_pvalues.png',dpi=200)